# Web scrapping- data to `bridge app`

## Short description
In our project we try to create app which enable to see statistics and progress of selected player. Additionally, we can follow rating of particular player and compare it to other.

## Creating data frame of played boards

In the first step we create data frame where we store data of played boards. Boards are from `KMP`tournament- taking place every month with about 500 pairs.

To get this data we can use `pandas` library, which enable easy web scrapping. Firstly we have to create some pipelines to structure data.

In [10]:
import pandas as pd
import requests
import math
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_hist(vector,n=30):
    pd.read_html(requests.get(f"https://r.bridgespider.com/{vector[0]}/prot/{1}/").content)[-1].to_csv("mydata.csv")
    tmp=pd.read_csv('mydata.csv')
    tmp['nr_rozdania']=1
    tmp['nr_turnieju']=vector[0]
    for j in range (len(vector)):
        for i in range(1,n):
            pd.read_html(requests.get(f"https://r.bridgespider.com/{vector[j]}/prot/{i+1}/").content)[-1].to_csv("mydata.csv")
            zz=pd.read_csv('mydata.csv')
            zz['nr_rozdania']=i+1
            zz['nr_turnieju']=vector[j]
            tmp=pd.concat([tmp,zz])
    return tmp

In [4]:
def pipeline(df2):
    df2=df2.drop(['Unnamed: 0','ośrodek','runda','wist','Unnamed: 11','lew','kontrakt'],axis=1)
    df2['rozg.']=df2['rozg.'].replace(['E','W'],'EW')
    df2['rozg.']=df2['rozg.'].replace(['N','S'],'NS')
    df2['zapis_NS']=df2['zapis']
    df2['zapis_EW']=df2['zapis']*(-1)
    df2=df2.drop('zapis',axis=1)
    return df2

In [5]:
def pipeline2(df_x):
    tmp1=df_x.copy()
    tmp2=df_x.copy()
    tmp1=tmp1.drop(['EW','% EW','zapis_EW'],axis=1)
    tmp1['linia']='NS'
    tmp2=tmp2.drop(['NS','% NS','zapis_NS'],axis=1)
    tmp2['linia']='EW'
    tmp1.columns=['id','rozg','wynik','nr_rozdania','nr_turnieju','zapis','linia']
    tmp2.columns=['id','rozg','wynik','nr_rozdania','nr_turnieju','zapis','linia']
    tmp=pd.concat([tmp1,tmp2])
    return tmp

In [6]:
def refactor(history):
    history=pipeline(history)
    history=pipeline2(history)
    return history

These 4 functions allow us to create our first data frame. Unfortunately, we have only id unique for particular tournament. Now we need data frame with names of these players. To get access to the data we have to use once again web scrapping tools. After this action we merge these 2 data frames and receive data set with almost unique id (name and surname).

## Create data frame with scores and names

In [11]:
def scores_pipeline(url):
    pd.read_html(requests.get(url).content)[-1].to_csv("mydata.csv")
    df=pd.read_csv('mydata.csv')
    df=df.drop(['Unnamed: 0','+/-','pkl', 'pdf'],axis=1)
    return df

In [12]:
def regex_trans_scores(df):
    import re
    pierwszy=[]
    drugi=[]
    for i in range(len(df)):
        pierwszy.append(re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[0]+re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[1])
        drugi.append(re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[2]+re.findall('[A-ZŻŹĆĄŚĘŁÓŃ][^A-ZŻŹĆĄŚĘŁÓŃ]*', df.nazwiska[i])[3])
    df['nazwisko_1']=pierwszy
    df['nazwisko_2']=drugi
    df=df.drop('nazwiska',axis=1)


    wk1=[]
    wk2=[]
    for i in range(len(df)):
        wk1.append(re.findall("\d+\.\d", df.wk[i])[0])
        wk2.append(re.findall("\d+\.\d", df.wk[i])[1])
    df['wk_1']=wk1
    df['wk_2']=wk2
    df=df.drop(['wk','ośrodek','okręg'],axis=1)
    return df

In [14]:
def score_refactor(scores,tab):
    url="https://r.bridgespider.com/"
    for i in tab:
        url=url+str(i)+'/'
        tmp=scores_pipeline(url)
        tmp=regex_trans_scores(tmp)
        tmp['nr_turnieju']=i
        scores=pd.concat([tmp,scores])
    return scores

In [17]:
def pipeline3(scores):
    scores_1=scores[['m-ce', 'nr', '%', 'nazwisko_1','wk_1','nr_turnieju']]
    scores_2=scores[['m-ce', 'nr', '%', 'nazwisko_2','wk_2','nr_turnieju']]
    
    scores_1['nazwisko']=scores_1['nazwisko_1']
    scores_2['nazwisko']=scores_2['nazwisko_2']
    
    scores_1['wk']=scores_1['wk_1']
    scores_2['wk']=scores_2['wk_2']
    
    scores_1=scores_1.drop(['nazwisko_1','wk_1'],axis=1)
    scores_2=scores_2.drop(['nazwisko_2','wk_2'],axis=1)
    
    scores_new=pd.concat([scores_1,scores_2])
    return scores_new

Now we can merge data frame history and scores on `name`. However, we have to remember that it can be possible that exist 2 players with the same name and surname. We can't manage with this fact because of not having unique id for every player. Our solution is to delete all players with same name and surname. It's a simplification but not so meningful.

## Players data frame
Now we download data frame with all players who played at least once in tournament. From this table we get `PID`- unique id. Then we can merge on this number instead of merging on name and surname.

In [19]:
def creating_players_df():
    url="https://msc.com.pl/cezar/?p=51&rok=0&page=1"
    pd.read_html(requests.get(url).content)[-1].to_csv("mydata.csv")
    df=pd.read_csv('mydata.csv')
    tables = pd.read_html(url,match="klub / drużyna")
    df=tables[2]
    for i in range(337):
        url=f"https://msc.com.pl/cezar/?p=51&rok=0&page={i}"
        tables = pd.read_html(url,match="klub / drużyna")
        df=pd.concat([df,tables[2]])
    return df

In [21]:
def merging_drop_duplicate_names(history,scores,df):
    df.rename(columns = {1: 'PID'}, inplace = True)
    df.rename(columns = {2: 'name'}, inplace = True)
    
    scores=scores.merge(df[['PID','name']],left_on='nazwisko',right_on='name')
    scores=scores.drop(['name'],axis=1)
    
    duplicates=scores[scores[['nr_turnieju','nazwisko']].duplicated(keep=False)]
    scores=pd.concat([scores,duplicates]).drop_duplicates(keep=False)
    return scores,history.merge(scores, left_on=['id','nr_turnieju'],right_on=['nr','nr_turnieju'])

## Progress of player
Other important feature of our app is possibilty to check progress of player. Progress means for us final results of all played tournaments. That's why we have to create one more data frame which contains `PID` of player and all results.

In [88]:
import numpy as np

In [8]:
def players_progress(history):
    vector=history['PID'].unique()
    df_x=pd.DataFrame(index=np.arange(len(vector)), columns=np.arange(21))
    df_x.columns=['PID',25890,25845,25809,25773,25737,25692,24384,24339,24303,24267,24222,24186,24150,22079 ,22043 ,21800,21764,21728,17693,17666]
    tab=[25890,25845,25809,25773,25737,25692,24384,24339,24303,24267,24222,24186,24150,22079 ,22043 ,21800,21764,21728,17693,17666]
    k=0
    for i in range(len(vector)):
        tmp=history[history['PID']==vector[i]][['PID','%','nr_turnieju']].drop_duplicates()
        df2=pd.DataFrame(index=np.arange(1), columns=np.arange(2))
        for j in range(20):
            df2.loc[j,0]=tab[j]

        df2=df2.rename({0:'nr_turnieju'},axis=1)
        df2=df2.merge(tmp,on='nr_turnieju',how='left')

        vec=np.array([vector[i]])
        vec2=df2['%'].to_numpy()
        df_x.iloc[i,:]=np.concatenate((vec,vec2),axis=0)
        k=k+1
    return df_x

## Statistics of particular players
Our large data frame include now only scores from all played boards. We need some additional statistics like average result but with specification if player defends or declares. The solution is to separate boards to some categories and add number of columns. 

### Separate to declarers and defenders

In [23]:
def separating_def_dec(history):
    declarers=history[history['rozg']==history['linia']]
    defenders=history[history['rozg']!=history['linia']]
    declarers['czy_rozgrywa']=True
    defenders['czy_rozgrywa']=False
    history=pd.concat([declarers,defenders])
    return history

### Separate to kinds of contracts- (częściowka-partial, końcówka, slam or above)
We also need here simplification because of not having access to details of boards. However, having number of points get by declarer (only positive number) we can almost surely predict kind of contract. Additionally, we create category for boards when declarer loses (`Wpadka`) and category for defenders when they achieve positive result (`Kokosy_na_Obronie`).

In [24]:
def separating_contracts(history):
    tmp1=history[ (history['zapis']>0) & (history['czy_rozgrywa'] )]
    tmp1['zapis2']=pd.cut(history['zapis'], bins=[0, 300, 900, math.inf],labels=['częściówka','końcówka','szlemik'], include_lowest=True)
    tmp2=history[ (history['zapis']<0) & (history['czy_rozgrywa']==False )]
    tmp2['zapis2']=pd.cut(history['zapis'], bins=[-math.inf,-900,-300,0],labels=['szlemik','końcówka','częściówka'], include_lowest=True)
    tmp3=history[ (history['zapis']>=0) & (history['czy_rozgrywa']==False )]
    tmp3['zapis2']='Kokosy_na_Obronie'
    tmp4=history[ (history['zapis']<=0) & (history['czy_rozgrywa']==True )]
    tmp4['zapis2']='Wpadka'
    history=pd.concat([tmp1,tmp2,tmp3,tmp4])
    return history

Now we have data frame with results of all boards from selected tournaments. Now we can create new data frame with statistics of particular player. But firstly we need some groupby statements.

In [28]:
def stats_mean(history):
    stats2=history[history['czy_rozgrywa']==True].groupby(['PID'])['wynik'].mean().reset_index(name='średnia_na_rozgrywce')
    stats3=history[history['czy_rozgrywa']==False].groupby(['PID'])['wynik'].mean().reset_index(name='średnia_na_obronie')

    stat=stats2.merge(stats3,on='PID')

    stat4=history[history['zapis2']=="Kokosy_na_Obronie"].groupby(['PID'])['wynik'].mean().reset_index(name="Kokosy_na_Obronie")
    stat5=history[history['zapis2']=="częściówka"].groupby(['PID'])['wynik'].mean().reset_index(name="częściówka")
    stat6=history[history['zapis2']=="końcówka"].groupby(['PID'])['wynik'].mean().reset_index(name="końcówka")
    stat7=history[history['zapis2']=="szlemik"].groupby(['PID'])['wynik'].mean().reset_index(name="szlemik")
    stat8=history[history['zapis2']=="Wpadka"].groupby(['PID'])['wynik'].mean().reset_index(name="Wpadka")


    stat=stat.merge(stat4,on='PID')
    stat=stat.merge(stat5,on='PID')
    stat=stat.merge(stat6,on='PID')
    stat=stat.merge(stat7,on='PID')
    stat=stat.merge(stat8,on='PID')
    return stat

In [29]:
def stats_count(history):
    stats2=history[history['czy_rozgrywa']==True].groupby(['PID'])['wynik'].count().reset_index(name='liczba_rozdan_na_rozgrywce')
    stats3=history[history['czy_rozgrywa']==False].groupby(['PID'])['wynik'].count().reset_index(name='liczba_rozdan_na_obronie')
    stat4=history[history['zapis2']=="Kokosy_na_Obronie"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_Kokosy_na_Obronie")
    stat5=history[history['zapis2']=="częściówka"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_częściówka")
    stat6=history[history['zapis2']=="końcówka"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_końcówka")
    stat7=history[history['zapis2']=="szlemik"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_szlemik")
    stat8=history[history['zapis2']=="Wpadka"].groupby(['PID'])['wynik'].count().reset_index(name="liczba_rozdan_Wpadka")
    
    stat=stats2.merge(stats3,on='PID')
    stat=stat.merge(stat4,on='PID')
    stat=stat.merge(stat5,on='PID')
    stat=stat.merge(stat6,on='PID')
    stat=stat.merge(stat7,on='PID')
    stat=stat.merge(stat8,on='PID')
    return stat

We can improve average scores using something like min-max scaler. In this way we compare players to each other. For example a lot of players have score of `Wpadka` equals about 30%. After using our scaler difference beetween players with 30% and 39% is not miserable 9 % but more meaningful.

In [50]:
def stats_percentile(df_stats):
    df_score_upd=pd.DataFrame({'PID':df_stats['PID']})
    vector=['średnia_na_rozgrywce', 'średnia_na_obronie', 'Kokosy_na_Obronie', 'częściówka', 'końcówka', 'szlemik', 'Wpadka']
    for i in vector:
        vec=df_stats[i]
        wektor=[]
        for j in range(len(vec)):
            wektor.append(round((vec[vec<vec[j]].count()/len(vec))*100,2))
        df_score_upd[i]=wektor
    return df_score_upd

Additionally we want to generate ranking of players according to `df_stats`. Which variable should be the most important one? In our opinion, the user should decide about that. So one of input argument allow user to select the ranking's feature. 

In [123]:
def generate_ranking(feature,stats): 
    stats=stats[['PID','name',feature]]
    return stats.sort_values(by=feature, ascending=False)

Other idea for creating ranking:
- we got number of tournament results let's say $n$. We can use a weighted average to prepare ranking. The weights would be counted in this way: 
    - last tournament weight=1, previous tournament $\frac{n-1}{n}$, then $\frac{n-2}{n}$ and so on. Last tournament's weight in ranking would be equals $\frac{1}{n}$
    - we divide (sum of results multipled by weights) by sum of weight
    $$\frac{\sum_{j=1}^{n} W_{i}*X{i}}{\sum_{j=1}^{n} W_{i}}   $$
$W_{i}$ - weight of i-th tournament<br>
$X_{i}$ - result of i-th tournament<br>
$n$ - number of played tournaments
- in our opinion is quite fair; players who played only one tournament for example one year ago even with good score don't take the high position
- ranking informs us about progress as well

In [115]:
def new_ranking_generator(players_progress,players):
    df=pd.DataFrame()
    df['PID']=players_progress['PID']
    final=np.array([])
    for j in range(len(players_progress)):
        vec=players_progress.fillna(0).loc[j, :].values.tolist()[1:]
        indexes=[z for z, x in enumerate(vec) if x !=0]
        weights=np.round(np.arange(1, 0, -0.05).tolist(),2)
        suma=0
        for i in indexes:
            suma+=weights[i]
        final=np.append(final,np.sum(np.dot(vec,weights))/suma)
    df['ranking']=final
    df=df.sort_values(by='ranking',ascending=False)
    df['ranking']=round(df['ranking'],2)
    df['PID']= df['PID']. astype (int)
    df['PID']= df['PID']. astype (str)
    wektor=[]
    for v in df['PID']:
        wektor.append(v.rjust(5,'0'))
    df['PID']=wektor
    df=df.merge(players[['PID','name']],on='PID')
    return df

After this part we have all data frames which we need. Short description:
- `df`: data frames with all players from Polish Bridge Union (`PZBS`)
- `history`: all boards played in selected tournaments
- `scores`: final scores from particular tournament
- `df_stats`: all numbers in cells tells us about percentile of selected player. For example result `50` means that this
player has better result than 50% of all players. Next variables are number of boards played in particular category.

Obviously, we want to easy run this script. In the last part, we write final version of functions, which enable to get data frame which we want.

## Function to get final version of df (players names and id)

In [ ]:
df=creating_players_df()  # you need a little bit of time to get results

## Function to get first version of scores

In [79]:
# input: tab_add is array with ids (except tournament with id 25890 it's set default) of tournaments which you want to select (you can get this number from URL).
def first_scores(tab_add,n=25890):
    scores=scores_pipeline("https://r.bridgespider.com/25890/")
    scores=regex_trans_scores(scores)
    scores['nr_turnieju']=n  #create instance for 1 tournament
    scores=score_refactor(scores,tab_add)
    scores=pipeline3(scores)
    return scores

## Function to get final version of history and final version of scores

In [66]:
# input: tab_test- exactly like tab_add but you must add also `25890` tournament
#        scores- output of `first_scores()`
#        df- output of `creating_players_df()`
def final_history_scores(tab_test,scores,df):
    history=get_hist(tab_test)
    history=refactor(history)
    scores,history=merging_drop_duplicate_names(history,scores,df)
    history=separating_def_dec(history)
    history=separating_contracts(history)
    return history

## Function to get final version of df_stats

In [110]:
# input: like above
def final_stats(history,df):
    
    df_stats=stats_mean(history).merge(stats_count(history),on='PID')
    df_stats_new=stats_percentile(df_stats)
    df_stats_new=df_stats_new.merge(df[['PID','name']],on='PID')

    df_stats_new=stats_percentile(df_stats)
    df_stats_new=df_stats_new.merge(df_stats[['PID','liczba_rozdan_na_rozgrywce', 'liczba_rozdan_na_obronie',
       'liczba_rozdan_Kokosy_na_Obronie', 'liczba_rozdan_częściówka',
       'liczba_rozdan_końcówka', 'liczba_rozdan_szlemik',
       'liczba_rozdan_Wpadka']],on='PID')
    df_stats=df_stats_new.merge(df[['PID','name']],on='PID')
    
    return df_stats

## Calling functions
In last step we should create 2 arrays:
- `tab_add` containing all selected tournaments ids except `25890`(default) 
- `tab_test` containg all selected tournaments ids with `25890`(default)
To get to know ids of tournaments, you should use https://bridgespider.com/ website, click on interesting 
you tournament (preffered are tournament calles as `KMP`) and get from URL number between 2 slashes.

## Example
I want to get scores from June and May edition of `KMP` tournament. June's id is `25890` (default selected) and May's id is `25845` (get from URL). 

In [71]:
tab_add=[25845]
tab_test=[25890,25845]

---------------------

In [ ]:
df=creating_players_df()  # you need a little bit of time to get results

In [70]:
first_scores=first_scores(tab_add) # it's only useful to call next function- quite bad I know

In [72]:
history=final_history_scores(tab_test,first_scores,df)

In [111]:
stats=final_stats(history,df)

In [11]:
players_progress=players_progress(history)

In [118]:
new_ranking=new_ranking_generator(players_progress,df)

-------

## Results

In [12]:
df

,0,PID,name,3,4,5,6,7,8
0,lokata,PID,imię i nazwisko,WZBS,WK,TK,PKL,liga,klub / drużyna
1,1.,06706,Apolinary Kowalski,WM,24,AS,189 039,E,VITAS ESBS Elblag
2,2.,08513,Piotr Gawryś,DS,24,AS,152 444,E,BRIDGESCANNER KS AZS Wratislavia I WR
3,3.,06658,Jacek Kalita,SW,24,AS,146 929,E,SPS CONSTRUCTION Kielce
4,4.,01221,Cezary Balicki,DS,24,AS,143 379,-,NaN
...,...,...,...,...,...,...,...,...,...
17233,NaN,15971,Bartosz Warwas,LD,0,D,1,-,NaN
17234,NaN,08612,Artur Wasiluk,PD,0,D,1,-,NaN
17235,NaN,14968,Tomasz Wąs,MA,0,D,1,-,NaN
17236,NaN,19826,Natalia Wąsowska,MA,0,D,1,-,NaN


In [77]:
scores

,m-ce,nr,%,nr_turnieju,nazwisko,wk,PID
0,1,517,68.83,25845,Lechosław Błaziński,12.0,05900
1,223,504,51.41,25890,Lechosław Błaziński,12.0,05900
2,2,303,67.16,25845,Michał Bobrykow,3.0,04064
3,431,289,43.48,25890,Michał Bobrykow,3.0,04064
4,3,237,67.03,25845,Radosław Rozman,2.5,18801
...,...,...,...,...,...,...,...
2108,512,452,36.05,25890,Janusz Kopras,4.0,12435
2109,517,438,34.02,25890,Stanisław Kowalczewski,1.5,17296
2110,519,356,32.97,25890,Janusz Bakalarz,0.5,19873
2111,520,361,31.28,25890,Jan Luber,0.0,21538


In [74]:
history

,id,rozg,wynik,nr_rozdania,nr_turnieju,zapis,linia,m-ce,nr,%,nazwisko,wk,PID,czy_rozgrywa,zapis2
38,8,EW,93.40,8,25890,420.0,EW,485,8,39.61,Bogdan Sadowski,1.5,13237,True,końcówka
39,8,EW,93.40,8,25890,420.0,EW,485,8,39.61,Stanisław Gołdyn,2.0,13843,True,końcówka
48,8,EW,63.39,28,25890,110.0,EW,485,8,39.61,Bogdan Sadowski,1.5,13237,True,częściówka
49,8,EW,63.39,28,25890,110.0,EW,485,8,39.61,Stanisław Gołdyn,2.0,13843,True,częściówka
50,8,EW,55.34,29,25890,600.0,EW,485,8,39.61,Bogdan Sadowski,1.5,13237,True,końcówka
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50997,474,EW,14.23,2,25845,-100.0,EW,417,474,44.11,Katarzyna Stachowiak,1.5,21060,True,Wpadka
51020,474,EW,16.34,14,25845,-50.0,EW,417,474,44.11,Piotr Surdyk,4.0,15044,True,Wpadka
51021,474,EW,16.34,14,25845,-50.0,EW,417,474,44.11,Katarzyna Stachowiak,1.5,21060,True,Wpadka
51028,474,EW,60.67,18,25845,-100.0,EW,417,474,44.11,Piotr Surdyk,4.0,15044,True,Wpadka


In [113]:
stats

,PID,średnia_na_rozgrywce,średnia_na_obronie,Kokosy_na_Obronie,częściówka,końcówka,szlemik,Wpadka,liczba_rozdan_na_rozgrywce,liczba_rozdan_na_obronie,liczba_rozdan_Kokosy_na_Obronie,liczba_rozdan_częściówka,liczba_rozdan_końcówka,liczba_rozdan_szlemik,liczba_rozdan_Wpadka,name
0,01001,76.76,63.65,48.78,45.00,43.11,99.59,96.76,14,13,5,10,6,1,5,Andrzej Ankiersztejn
1,01006,52.43,48.11,40.95,22.16,80.27,95.00,71.35,17,10,5,9,6,1,6,Zdzisław Baryła
2,01024,32.43,69.19,62.70,82.43,51.08,79.59,36.89,32,25,7,19,14,2,15,Władysław Chodziński
3,01037,33.78,21.08,22.03,32.43,59.05,53.65,52.97,33,24,6,15,19,3,14,Stanisław Drobot
4,01056,0.27,18.51,16.76,14.46,5.00,21.22,4.05,11,19,5,10,8,1,6,Grażyna Blech
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,21959,68.92,35.14,42.43,92.84,35.00,48.92,23.24,24,26,9,13,18,2,8,Wiesław Wójcik
736,21971,68.24,4.32,4.59,30.41,14.59,55.81,88.38,25,32,6,20,23,2,6,Ewa Powała
737,21972,68.24,4.32,4.59,30.41,14.59,55.81,88.38,25,32,6,20,23,2,6,Sławomir Powała
738,22073,19.86,63.38,98.65,18.78,0.68,92.70,34.59,8,18,5,10,7,1,3,Waldemar Rozmarynowski


In [96]:
players_progress

,PID,25890,25845,25809,25773,25737,25692,24384,24339,24303,...,24222,24186,24150,22079,22043,21800,21764,21728,17693,17666
0,13237,39.61,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,13843,39.61,40.25,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,01257,54.15,54.51,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,01354,54.15,54.51,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,01252,64.6,57.81,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,06117,nan,51.39,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1208,10268,nan,41.9,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1209,21511,nan,41.9,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1210,18367,nan,43.78,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [119]:
new_ranking

,PID,ranking,name
0,06115,68.83,Radosław Szopiński
1,19705,67.03,Jan Syska
2,07902,65.97,Damian Mazurak
3,02131,64.47,Ryszard Smejda
4,02185,64.47,Damian Wroński
...,...,...,...
1207,01953,30.49,Eugeniusz Chwalko
1208,20987,30.33,Sylwia Kruszyńska
1209,20986,30.33,Stella Kruszyńska
1210,21472,24.82,Małgorzata Skowrońska


Obviously we don't want to call these functions every time. We can save data frames to `.csv files`. So we need to write 2 more functions.

In [120]:
def save_to_csv(df,history,stats,scores,players_progress,new_ranking):
    df.to_csv("df.csv",index=False)
    history.to_csv("history.csv",index=False)
    stats.to_csv("stats.csv",index=False)
    scores.to_csv("scores.csv",index=False)
    players_progress.to_csv("players_progress.csv",index=False)
    new_ranking.to_csv("new_ranking.csv",index=False)

In [5]:
def get_from_csv():
    df=pd.read_csv("df.csv",index_col=False)
    history=pd.read_csv("history.csv",index_col=False)
    stats=pd.read_csv("stats.csv",index_col=False)
    scores=pd.read_csv("scores.csv",index_col=False)
    #players_progress=pd.read_csv("players_progress.csv",index_col=False)
    #new_ranking=pd.read_csv("new_ranking.csv",index_col=False)
    return df,history,stats,scores,players_progress,new_ranking

## Finish?
Now we can easily create data frames which we want to run our `bridge app`. However, looking only at history data can be quite boring. We need also function which synchronises archive data frames with new results. The quickest way is to concatenate archive with fresh results (`scores` and `history`). Having new version of data we can update `stats` and `players_progress`. 

In [78]:
def concatenate_scores_or_history(df_archive,df_fresh):
    return pd.concat([df_archive,df_fresh])

To get the new stats we have to call `final_stats()` once again with changed arguments.

## Conclusion
To sum up, we know that we use some simplification in our app. It's caused of not having access to some data. The problem can also be the `df` with players. It should be quite oft updating to not skip rising bridge star. In this moment we finish our work with `.csv files` and start to develop app, which perfectly visualise our idea.

## Appendix
In appendix we focus on generating ranking of players. We don't need to crete any additionall data frames. We use function `generate_ranking()`. This function is responsible only for selecting and sorting data but output would be quite helpful during developping the app.

In [125]:
def generate_sort_ranking_df(stats):

    vector=['średnia_na_rozgrywce', 'średnia_na_obronie', 'Kokosy_na_Obronie', 'częściówka', 'końcówka', 'szlemik', 'Wpadka']
    
    for v in vector:
        df=generate_ranking(v,stats)
        df.to_csv(f'{v}_sort_df.csv',index=False)

In [126]:
generate_sort_ranking_df(stats)